# Create Azure Machine Learning Compute

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../jobs/configuration.ipynb) 
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Create Azure Machine Learning Compute Instance from Python SDK
- Create Azure Machine Learning Compute Cluster from Python SDK

**Motivations** - Azure Machine Learning needs compute to run tasks. This tutorial will introduce you to create compute for machine learning

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the compute will be created.

## 1.1. Import the required libraries

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import ComputeInstance, AmlCompute
from azure.identity import DefaultAzureCredential

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
# Enter details of your AML workspace
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
# get a handle to the workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# 2 Create Compute Instance
An Azure Machine Learning [compute instance](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-instance) is a managed cloud-based workstation for data scientists.

Compute instance can be created using the `ComputeInstance` class. This class accepts the following key parameters:
- `name` - Name of the compute.
- `description` - Description of the compute
- `size` - The VM size to use for the compute instance. For more information, see [Supported VM series and sizes](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target#supported-vm-series-and-sizes). 

## 2.1 Create a minimal Compute Instance
In this example we will create a compute instance with only the `name` specified.

In [ ]:
# Compute Instances need to have a unique name across the region.
# Here we create a unique name with current datetime
from azure.ai.ml.entities import ComputeInstance, AmlCompute
import datetime

ci_minimal_name = "min-ci" + datetime.datetime.now().strftime("%Y%m%d%H%M")

ci_minimal = ComputeInstance(name=ci_minimal_name)
ml_client.begin_create_or_update(ci_minimal).result()

## 2.2 Create a Compute Instance of specific size
In this example, we will specify the size of the VM. For more information, see [Supported VM series and sizes](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target#supported-vm-series-and-sizes). 

In [ ]:
# Compute Instances need to have a unique name across the region.
# Here we create a unique name with current datetime
from azure.ai.ml.entities import ComputeInstance, AmlCompute
import datetime

ci_basic_name = "basic-ci" + datetime.datetime.now().strftime("%Y%m%d%H%M")
ci_basic = ComputeInstance(name=ci_basic_name, size="STANDARD_DS3_v2")
ml_client.begin_create_or_update(ci_basic).result()

## 2.3 Interact with a Compute Instance
In this below cells, we will specify how you can perform basic operations on a Compute Instance. A full list of operations can be found here. Note that the below operations are applicable for all computes (not just compute instances).

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

# Get compute
ci_basic_state = ml_client.compute.get(ci_basic_name)

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

# Stop compute
ml_client.compute.begin_stop(ci_basic_name).wait()

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

# Start compute
ml_client.compute.begin_start(ci_basic_name).wait()

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

# Restart compute
ml_client.compute.begin_restart(ci_basic_name).wait()

## 2.3 Clean up Resources
Clean up the Compute Instances to save costs

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

ml_client.compute.begin_delete(ci_minimal_name).wait()

In [ ]:
from azure.ai.ml.entities import ComputeInstance, AmlCompute

ml_client.compute.begin_delete(ci_basic_name).wait()

# 3. Create Compute Cluster
Azure Machine Learning compute cluster is a managed-compute infrastructure that allows you to easily create a single or multi-node compute. 

A compute cluster can be created using the `AmlCompute` class. Some of the key parameters of this class are:
- `name` - Required. Name of the compute.		
- `description` - Description of the compute.		
- `location` - The location for the compute. If omitted, defaults to the workspace location.
- `size` - The VM size to use for the cluster. For more information, see [Supported VM series and sizes](https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target#supported-vm-series-and-sizes).
- `tier` - The VM priority tier to use for the cluster. The priority can be `dedicated` or `low_priority`. If not specified, a `dedicated` cluster will be created.
- `min_instances` - The minimum number of nodes to use on the cluster. Setting the minimum number of nodes to 0 allows Azure ML to autoscale the cluster down to zero nodes when not in use. Any value larger than 0 will keep that number of nodes running, even if the cluster is not in use. Default is 0.
- `max_instances` - The maximum number of nodes to use on the cluster. Default is 1.
- `idle_time_before_scale_down` - Node idle time in seconds before scaling down the cluster. Default is 120 seconds

## 3.1 Create a basic Compute Cluster
In this example we will create a basic compute cluster with a minimal set of parameters.


In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_basic = AmlCompute(
    name="basic-example",
    type="amlcompute",
    size="STANDARD_DS3_v2",
    location="westus",
    min_instances=0,
    max_instances=2,
    idle_time_before_scale_down=120,
)
ml_client.begin_create_or_update(cluster_basic).result()

## 3.2 Create a low priority cluster
In this example we will create a low priority cluster instead of the default dedicated cluster.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_low_pri = AmlCompute(
    name="low-pri-example",
    size="STANDARD_DS3_v2",
    min_instances=0,
    max_instances=2,
    idle_time_before_scale_down=120,
    tier="low_priority",
)
ml_client.begin_create_or_update(cluster_low_pri).result()